In [ ]:
conda install tensorflow

In [ ]:
conda install keras

In [3]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report as cr
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.neural_network import MLPClassifier

E:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Load dataset and split them into train and test set

In [2]:
path = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/data/train_set/'
data_unsplit = pd.read_csv(path + "data_unsplit.csv")
X_dis=data_unsplit.iloc[:,1:-1]
y_dis=data_unsplit["emotion_idx"]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_dis = scaler.fit_transform(X_dis)

train_x_dis,test_x_dis,train_y_dis,test_y_dis=train_test_split(X_dis,y_dis,test_size=0.2,random_state=3662)

# keras

In [4]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### Use GridsearchCV to determine Parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
MLP_parm_gs = {
    'hidden_layer_sizes': [(1000,), (1000,1000), (2000,), (2000,1000), (3000,), (1000,2000), (1000,1000,1000)],
    'learning_rate':['adaptive', 'constant'],
    'solver':['lbfgs','sgd','adam'],
    'early_stopping':[True] , 
    'validation_fraction':[0.2],
                        
}
MLP_model_gs = MLPClassifier()
MLP_gsearch = GridSearchCV(estimator = MLP_model_gs, 
                        param_grid = MLP_parm_gs, 
                        scoring ='accuracy',
                        cv = 3,
                        n_jobs = 10,
                        verbose=3,
                        return_train_score=True
                       )
MLP_gsearch.fit(train_x_dis,train_y_dis)

In [ ]:
from sklearn.metrics import accuracy_score 
print("Training dataset Accuracy")
train_preds = MLP_gsearch.predict(train_x_dis)
train_accuracy = accuracy_score(train_y_dis, train_preds) 
print(train_accuracy)

In [ ]:
print("Testing dataset Accuracy")
test_preds = MLP_gsearch.predict(test_x_dis)
test_accuracy = accuracy_score(test_y_dis, test_preds) 
print(test_accuracy)

In [ ]:
MLP_gsearch.best_params_

### Use K-Fold to estimate the accuracy (49.36%)

In [ ]:
MLP_model_final =MLPClassifier(early_stopping=True,
                              hidden_layer_sizes=(1000,1000),
                              learning_rate='constant',
                              solver='lbfgs',
                              validation_fraction=0.2)

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(logistic_model_final, X_dis,y_dis, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

### Fit the final model and save it

In [ ]:
t0 = time.time()
MLP_model_final.fit(X_dis,y_dis)
print("done in %0.3fs" % (time.time() - t0))

In [ ]:
joblib.dump(logistic_model_final,'MLP_model_final.m')

In [ ]:
#joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/MLP_model_final.m')

In [8]:
path = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/data/train_set/'
data_unsplit = pd.read_csv(path + "data_unsplit.csv")
X_dis=data_unsplit.iloc[:,1:-1]
y_dis=data_unsplit["emotion_idx"]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X= scaler.fit_transform(X_dis)

from keras.utils import to_categorical
Y = to_categorical(y_dis)
Y = Y[:,1:]

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=3662)

In [14]:
from keras.layers import Dense, Activation, Flatten, Input,Dropout
from keras.layers import BatchNormalization
from keras.models import Model
from keras import initializers
input_shape = [6006] 
input_layer = Input(input_shape)
x = BatchNormalization()(input_layer) 
x = Dense(400,activation='relu',kernel_initializer=initializers.glorot_normal(seed=None))(x)
x = Dropout(0.25)(x)
x = BatchNormalization()(x) 
x = Dense(200,activation='relu',kernel_initializer=initializers.glorot_normal(seed=None))(x)
x = Dropout(0.25)(x)
x = Dense(100,activation='relu',kernel_initializer=initializers.glorot_normal(seed=None))(x)
x = Dropout(0.25)(x)
output_layer = Dense(22,activation='softmax',kernel_initializer=initializers.glorot_normal(seed=None))(x) 
model = Model(input_layer,output_layer)
# model.summary()

In [15]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.0001),metrics=['accuracy'])
# model.fit(X_train,y_train,epochs=50, batch_size=200)

In [16]:
# model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.00001),metrics=['accuracy'])
model.fit(X_train,y_train,epochs=200, batch_size=200)
model.fit(X_train,y_train,epochs=200, batch_size=200)


Epoch 1/200
2000/2000 [==============================] - 3s 1ms/step - loss: 3.3418 - accuracy: 0.0705
Epoch 2/200
2000/2000 [==============================] - 1s 480us/step - loss: 2.9223 - accuracy: 0.1405
Epoch 3/200
2000/2000 [==============================] - 1s 642us/step - loss: 2.7005 - accuracy: 0.1925
Epoch 4/200
2000/2000 [==============================] - 1s 670us/step - loss: 2.5768 - accuracy: 0.2205
Epoch 5/200
2000/2000 [==============================] - 1s 685us/step - loss: 2.4747 - accuracy: 0.2545
Epoch 6/200
2000/2000 [==============================] - 1s 663us/step - loss: 2.3349 - accuracy: 0.2925
Epoch 7/200
2000/2000 [==============================] - 1s 695us/step - loss: 2.2553 - accuracy: 0.3145
Epoch 8/200
2000/2000 [==============================] - 2s 768us/step - loss: 2.1756 - accuracy: 0.3495
Epoch 9/200
2000/2000 [==============================] - 1s 657us/step - loss: 2.0960 - accuracy: 0.3570
Epoch 10/200
2000/2000 [==============================] -

2000/2000 [==============================] - 3s 1ms/step - loss: 0.5094 - accuracy: 0.8445
Epoch 80/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5505 - accuracy: 0.8205
Epoch 81/200
2000/2000 [==============================] - 3s 1ms/step - loss: 0.5508 - accuracy: 0.8125
Epoch 82/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5209 - accuracy: 0.8320
Epoch 83/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5240 - accuracy: 0.8350
Epoch 84/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5142 - accuracy: 0.8380
Epoch 85/200
2000/2000 [==============================] - 3s 1ms/step - loss: 0.5028 - accuracy: 0.8395
Epoch 86/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.5147 - accuracy: 0.8310
Epoch 87/200
2000/2000 [==============================] - 3s 1ms/step - loss: 0.4719 - accuracy: 0.8520
Epoch 88/200
2000/2000 [==============================] - 2s 1ms/step - loss:

2000/2000 [==============================] - 2s 1ms/step - loss: 0.1886 - accuracy: 0.9405
Epoch 158/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.1797 - accuracy: 0.9520
Epoch 159/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.1774 - accuracy: 0.9425
Epoch 160/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.1830 - accuracy: 0.9455
Epoch 161/200
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2017 - accuracy: 0.9395
Epoch 162/200
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1828 - accuracy: 0.9420
Epoch 163/200
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1707 - accuracy: 0.9515
Epoch 164/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.1791 - accuracy: 0.9480
Epoch 165/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.1786 - accuracy: 0.9445
Epoch 166/200
2000/2000 [==============================] - 2s 1ms/ste

2000/2000 [==============================] - 4s 2ms/step - loss: 0.0773 - accuracy: 0.9765
Epoch 36/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0827 - accuracy: 0.9750
Epoch 37/200
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0977 - accuracy: 0.9725
Epoch 38/200
2000/2000 [==============================] - 2s 954us/step - loss: 0.0894 - accuracy: 0.9755
Epoch 39/200
2000/2000 [==============================] - 2s 986us/step - loss: 0.0915 - accuracy: 0.9770
Epoch 40/200
2000/2000 [==============================] - 2s 964us/step - loss: 0.1076 - accuracy: 0.9690
Epoch 41/200
2000/2000 [==============================] - 3s 2ms/step - loss: 0.1057 - accuracy: 0.9670
Epoch 42/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.1041 - accuracy: 0.9705
Epoch 43/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0960 - accuracy: 0.9715
Epoch 44/200
2000/2000 [==============================] - 4s 2ms/step -

2000/2000 [==============================] - 4s 2ms/step - loss: 0.0504 - accuracy: 0.9825
Epoch 114/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0547 - accuracy: 0.9835
Epoch 115/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0675 - accuracy: 0.9770
Epoch 116/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0571 - accuracy: 0.9835
Epoch 117/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0584 - accuracy: 0.9785
Epoch 118/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0704 - accuracy: 0.9755
Epoch 119/200
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0630 - accuracy: 0.9800
Epoch 120/200
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0638 - accuracy: 0.9820
Epoch 121/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0556 - accuracy: 0.9820
Epoch 122/200
2000/2000 [==============================] - 4s 2ms/ste

2000/2000 [==============================] - 4s 2ms/step - loss: 0.0607 - accuracy: 0.9825
Epoch 192/200
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0493 - accuracy: 0.9880
Epoch 193/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0473 - accuracy: 0.9840
Epoch 194/200
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0485 - accuracy: 0.9855
Epoch 195/200
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0523 - accuracy: 0.9835
Epoch 196/200
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0430 - accuracy: 0.9885
Epoch 197/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0508 - accuracy: 0.9840
Epoch 198/200
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0454 - accuracy: 0.9845
Epoch 199/200
2000/2000 [==============================] - 3s 1ms/step - loss: 0.0391 - accuracy: 0.9890
Epoch 200/200
2000/2000 [==============================] - 3s 2ms/ste

In [17]:
from sklearn.metrics import accuracy_score 
a=np.argmax(model.predict(X_train), axis=1)
accuracy_score(np.argmax(y_train, axis=1), a) 

1.0

In [18]:
pred=np.argmax(model.predict(X_test), axis=1)
accuracy_score(np.argmax(y_test, axis=1), pred) 

0.542

In [13]:
kfold = KFold(n_splits=2, shuffle=True)
results = cross_val_score(model, X_dis,y_dis, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <keras.engine.training.Model object at 0x00000257E5BB8B48> does not.

In [ ]:
from keras.optimizers import Adam
start_time = time.time()
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.01),metrics=['accuracy'])
model.fit(X_train,y_train, validation_data=[X_test,y_test],epochs=50, batch_size=300)
print('lr=0.01, train complete')
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.001),metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=200, batch_size=250)
print('lr=0.001, train complete')
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.0001),metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=200, batch_size=200)
print('lr=0.0001, train complete')
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.00001),metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=200, batch_size=200)
print('lr=0.000001, train complete')
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.001),metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=200, batch_size=250)
print('lr=0.001, train complete')
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.0001),metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=200, batch_size=200)
print('lr=0.0001, train complete')
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.00001),metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=50, batch_size=200)
print('lr=0.0001, train complete')
print("--- %s seconds ---" % (time.time() - start_time))